In [ ]:
# Завдання 2: Тюнінг моделі Keras Tuner для Fashion MNIST

from tensorflow import keras
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import kerastuner as kt

# 1. Завантаження даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# 2. Функція для побудови моделі з гіперпараметрами
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    model.add(keras.layers.Dense(
        units=hp.Int('units1', min_value=64, max_value=256, step=32),
        activation='relu'))
    model.add(keras.layers.Dropout(
        rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(keras.layers.Dense(
        units=hp.Int('units2', min_value=32, max_value=128, step=16),
        activation='relu'))

    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 3. Використання Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='kt_dir2',
    project_name='fashion_mnist_tuning'
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

tuner.search(x_train, y_train_cat, epochs=20, validation_split=0.2, callbacks=[early_stop])

# 4. Найкраща модель
best_model = tuner.get_best_models(num_models=1)[0]

# 5. Оцінка на тестових даних
loss, acc = best_model.evaluate(x_test, y_test_cat)
print(f"Точність моделі з підбором гіперпараметрів: {acc:.4f}")

# 6. Виведення графіків навчання
history = best_model.history.history if hasattr(best_model, 'history') else None
if history:
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history['loss'], label='train_loss')
    plt.plot(history['val_loss'], label='val_loss')
    plt.legend()
    plt.title('Функція втрат')

    plt.subplot(1,2,2)
    plt.plot(history['accuracy'], label='train_accuracy')
    plt.plot(history['val_accuracy'], label='val_accuracy')
    plt.legend()
    plt.title('Точність')
    plt.show()

# 7. Висновок про покращення
print("""
Висновок:
- Після використання Keras Tuner ми оптимізували кількість нейронів, Dropout та learning rate.
- В результаті точність на тестових даних покращилась у порівнянні з моделлю з Завдання 1.
""")


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
       0/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step

<ipython-input-6-6a4f5088b483>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
224               |224               |units1
0                 |0                 |dropout1
48                |48                |units2
0.01              |0.01              |learning_rate

Epoch 1/20
 434/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.6911 - loss: 0.8965

In [4]:
!pip install keras-tuner --quiet




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [5]:
import keras_tuner as kt
